In [1]:
from dataclasses import dataclass
import numpy as np

@dataclass
class Params:
    s: float = 1.0      # company benefit per successful delivery
    e: float = 0.30     # rider self-disbursement cost
    d: float = 0.25     # rider effort-cost curvature
    q: float = 0.19     # company effort-cost curvature
    delta: float = 0.20 # effectiveness of rider's own effort (δ)
    eta: float = 0.75   # synergy coefficient (η), with (δ+η) <= 1
    V: float = 1.0      # max perceived loss (penalty channel)

# --------- Core helpers (payoffs & responses) ---------

def rho(a, b, p: Params):
    return p.delta * a + p.eta * a * b

def rider_payoff(a, b, f, p: Params):
    # Δ_DR = (f - e) * ρ(a,b) - d * a^2
    return (f - p.e) * rho(a, b, p) - p.d * a**2

def company_payoff(a, b, f, p: Params):
    # Δ_Q = (s - f) * ρ(a,b) - q * b^2
    return (p.s - f) * rho(a, b, p) - p.q * b**2

def rider_best_response_no_penalty(f, b, p: Params):
    # a* = (f - e) * (δ + η b) / (2d)
    return max(0.0, min(1.0, (f - p.e) * (p.delta + p.eta * b) / (2.0 * p.d)))

def company_best_response_no_penalty(f, a, p: Params):
    # b* = (s - f) * η a / (2q)
    return max(0.0, min(1.0, (p.s - f) * p.eta * a / (2.0 * p.q)))

# Penalty-channel helpers (multiplicative factor (1 - t/V))
def rider_best_response_with_penalty(f, b, t, p: Params):
    coef = (f - p.e - t) * (1.0 - t / p.V)
    return max(0.0, min(1.0, coef * (p.delta + p.eta * b) / (2.0 * p.d)))

def company_best_response_with_penalty(f, a, t, p: Params):
    coef = (p.s - f) * (1.0 - t / p.V)
    return max(0.0, min(1.0, coef * p.eta * a / (2.0 * p.q)))

# --------- Non-penalty equilibria ---------

def equilibrium_simultaneous_no_penalty(p: Params):
    # Use fixed-point best-response iteration across f grid and pick best f for company
    # Theory: f* = (s+e)/2 is optimal (concave Δ_Q in f) in non-penalty. :contentReference[oaicite:4]{index=4}
    f_star = 0.5 * (p.s + p.e)
    # Given f*, find (a,b) Nash in efforts via best responses
    a, b = 0.2, 0.2
    for _ in range(200):
        a = rider_best_response_no_penalty(f_star, b, p)
        b = company_best_response_no_penalty(f_star, a, p)
    return {
        "scenario": "no_penalty_simultaneous",
        "f_star": f_star,
        "a_star": a,
        "b_star": b,
        "rho": rho(a, b, p),
        "payoff_rider": rider_payoff(a, b, f_star, p),
        "payoff_company": company_payoff(a, b, f_star, p),
    }

def equilibrium_prior_commitment_no_penalty(p: Params):
    # Company chooses b and f (leader); rider best-responds a.
    # Paper indicates same f* = (s+e)/2 under prior commitment; company sets higher b than simultaneous. :contentReference[oaicite:5]{index=5}
    f_star = 0.5 * (p.s + p.e)
    # Choose b to maximize company payoff anticipating a(a|b,f*)
    b_grid = np.linspace(0, 1, 1001)
    best = None
    for b in b_grid:
        a = rider_best_response_no_penalty(f_star, b, p)
        val = company_payoff(a, b, f_star, p)
        if (best is None) or (val > best[0]):
            best = (val, a, b)
    val, a_star, b_star = best
    return {
        "scenario": "no_penalty_prior_commitment",
        "f_star": f_star,
        "a_star": a_star,
        "b_star": b_star,
        "rho": rho(a_star, b_star, p),
        "payoff_rider": rider_payoff(a_star, b_star, f_star, p),
        "payoff_company": val,
    }

def equilibrium_prior_commitment_with_sharing(p: Params, P: float):
    # Sharing rider effort cost: rider's effective cost curvature becomes (1-P)*d; company also pays P*d*a^2.
    # We adapt best response & company payoff accordingly. (Matches the 'sec' setup.) :contentReference[oaicite:6]{index=6}
    f_star = None
    # According to the paper, fee falls with higher sharing P; we’ll optimize f numerically in [e, s].
    f_grid = np.linspace(p.e, p.s, 401)
    best = None
    for f in f_grid:
        # leader chooses b; rider best-responds with modified cost
        b_grid = np.linspace(0, 1, 201)
        for b in b_grid:
            a = max(0.0, min(1.0, (f - p.e) * (p.delta + p.eta * b) / (2.0 * ((1 - P) * p.d + 1e-12))))
            # Company payoff now includes - q b^2 - P d a^2
            rho_ab = rho(a, b, p)
            payoff_company = (p.s - f) * rho_ab - p.q * b**2 - P * p.d * a**2
            if (best is None) or (payoff_company > best[0]):
                best = (payoff_company, f, a, b)
    payoff_Q, f_star, a_star, b_star = best
    payoff_r = (f_star - p.e) * rho(a_star, b_star, p) - (1 - P) * p.d * a_star**2
    return {
        "scenario": f"no_penalty_prior_commitment_sharing_P={P:.2f}",
        "f_star": f_star,
        "a_star": a_star,
        "b_star": b_star,
        "rho": rho(a_star, b_star, p),
        "payoff_rider": payoff_r,
        "payoff_company": payoff_Q,
    }

# --------- With-penalty (prior commitment on b; company sets (f, t)) ---------

def equilibrium_with_penalty_prior_commitment(p: Params, f_bounds=None, t_bounds=None):
    # Paper shows: effort responds positively to f up to a point; higher t reduces efforts for fixed f.
    # We do a small bilevel optimization: leader chooses (f,t,b), follower chooses a.
    # We grid-search (f,t), and for each choose b to maximize company payoff given rider best-response a.
    # See penalty rationale & probability factor (1 - t/V). :contentReference[oaicite:7]{index=7} :contentReference[oaicite:8]{index=8}
    if f_bounds is None:
        f_bounds = (p.e, p.s)  # reasonable range
    if t_bounds is None:
        t_bounds = (0.0, p.V)
    f_grid = np.linspace(*f_bounds, 121)
    t_grid = np.linspace(*t_bounds, 121)
    best = None
    for f in f_grid:
        for t in t_grid:
            # company picks b (prior commit) to maximize its payoff anticipating a
            b_grid = np.linspace(0, 1, 161)
            for b in b_grid:
                a = rider_best_response_with_penalty(f, b, t, p)
                # Δ_Q^t = (s - f) * ρ(a,b) * (1 - t/V) - q b^2
                factor = (1.0 - t / p.V)
                val_Q = (p.s - f) * rho(a, b, p) * factor - p.q * b**2
                if (best is None) or (val_Q > best[0]):
                    # Δ_DR^t = (f - e - t) * ρ(a,b) * (1 - t/V) - d a^2
                    val_R = (f - p.e - t) * rho(a, b, p) * factor - p.d * a**2
                    best = (val_Q, val_R, f, t, a, b)
    val_Q, val_R, f_star, t_star, a_star, b_star = best
    return {
        "scenario": "with_penalty_prior_commitment",
        "f_star": f_star,
        "t_star": t_star,
        "a_star": a_star,
        "b_star": b_star,
        "rho": rho(a_star, b_star, p),
        "payoff_rider": val_R,
        "payoff_company": val_Q,
    }

# --------- Example run ---------
if __name__ == "__main__":
    p = Params()  # defaults from paper’s illustrative calibration
    res_sim = equilibrium_simultaneous_no_penalty(p)
    res_prior = equilibrium_prior_commitment_no_penalty(p)
    res_share = equilibrium_prior_commitment_with_sharing(p, P=0.25)  # try 25% cost sharing
    res_pen = equilibrium_with_penalty_prior_commitment(p)

    for r in (res_sim, res_prior, res_share, res_pen):
        print(r["scenario"])
        for k, v in r.items():
            if k != "scenario":
                print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")
        print()


no_penalty_simultaneous
  f_star: 0.6500
  a_star: 0.2197
  b_star: 0.1517
  rho: 0.0689
  payoff_rider: 0.0121
  payoff_company: 0.0198

no_penalty_prior_commitment
  f_star: 0.6500
  a_star: 0.5096
  b_star: 0.7040
  rho: 0.3710
  payoff_rider: 0.0649
  payoff_company: 0.0357

no_penalty_prior_commitment_sharing_P=0.25
  f_star: 0.5992
  a_star: 0.7581
  b_star: 1.0000
  rho: 0.7202
  payoff_rider: 0.1078
  payoff_company: 0.0627

with_penalty_prior_commitment
  f_star: 0.6500
  t_star: 0.0000
  a_star: 0.5108
  b_star: 0.7063
  rho: 0.3727
  payoff_rider: 0.0652
  payoff_company: 0.0357

